In [1]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
path = ("./resources/Team_df.csv")
com_data = pd.read_csv(path)
print(com_data.shape)
com_data.head()

(224, 23)


,Team,Week,Result,Rec,Home,Opp,TmScore,OppScore,O_1stD,O_Tot_yd,...,D_1stD,D_Tot_Yd,D_P_Yd,D_R_Yd,D_TO,Offence,Defense,Sp.Tms,Prediction_LR,Prediction_ADA
0,Buffalo Bills,1,1,1-0,1,Chicago Bears,34.0,14.0,18.0,386.0,...,24.0,322.0,188.0,134.0,2.0,21.58,0.85,-0.52,0.418034,0.025844
1,Buffalo Bills,2,1,2-0,0,Indianapolis Colts,27.0,24.0,21.0,371.0,...,22.0,354.0,245.0,109.0,2.0,7.11,-1.29,-3.09,0.418034,0.025844
2,Buffalo Bills,3,1,3-0,1,Tampa Bay Buccaneers,34.0,24.0,22.0,407.0,...,27.0,446.0,411.0,35.0,1.0,20.56,-15.15,3.43,0.418034,0.025844
3,Buffalo Bills,4,0,3-1,1,Arizona Cardinals,20.0,37.0,24.0,401.0,...,27.0,465.0,249.0,216.0,1.0,8.94,-21.72,-2.64,0.418034,0.025844
4,Buffalo Bills,5,1,4-1,0,Seattle Seahawks,26.0,17.0,24.0,476.0,...,22.0,354.0,262.0,92.0,2.0,13.76,-6.71,-0.05,0.418034,0.025844


In [3]:
# get index of every team's data
team_index = com_data['Team']

# Remove Opponent, Score, Result
super_model_data = com_data[['Team', 'Opp', 'TmScore', 'O_1stD', 'O_Tot_yd', 'O_P_Yd', 'O_R_Yd', 'O_TO',
                         'D_1stD', 'D_Tot_Yd', 'D_P_Yd', 'D_R_Yd', 'D_TO', 'Home']]
#  , 'ADA_Pred_Mean', 'LOG_Prediction'

# change to season stats
season_stats = ['O_1stD', 'O_Tot_yd', 'O_P_Yd', 'O_R_Yd', 'O_TO',
                         'D_1stD', 'D_Tot_Yd', 'D_P_Yd', 'D_R_Yd', 'D_TO']

super_model_data[season_stats] = super_model_data[season_stats] * 16

# standardise the data
from sklearn import preprocessing

sd_data = ['O_1stD', 'O_Tot_yd', 'O_P_Yd', 'O_R_Yd', 'O_TO',
                         'D_1stD', 'D_Tot_Yd', 'D_P_Yd', 'D_R_Yd', 'D_TO']
#  , 'ADA_Pred_Mean', 'LOG_Prediction'

super_model_data[sd_data] = preprocessing.scale(super_model_data[sd_data])

#get indexs of every teams
team_index = com_data['Team']

super_model_data = pd.get_dummies(super_model_data)
super_model_data.head()

/opt/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/opt/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/dev/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

,TmScore,O_1stD,O_Tot_yd,O_P_Yd,O_R_Yd,O_TO,D_1stD,D_Tot_Yd,D_P_Yd,D_R_Yd,...,Opp_New Orleans Saints,Opp_New York Giants,Opp_New York Jets,Opp_Philadelphia Eagles,Opp_Pittsburgh Steelers,Opp_San Francisco 49ers,Opp_Seattle Seahawks,Opp_Tampa Bay Buccaneers,Opp_Tennessee Titans,Opp_Washington Football Team
0,34.0,-0.762952,0.208713,0.739647,-0.863009,-0.655336,0.856943,-0.162220,-0.528247,0.560785,...,0,0,0,0,0,0,0,0,0,0
1,27.0,-0.108993,0.010164,0.195539,-0.299168,0.465238,0.436910,0.236406,0.231372,0.034050,...,0,0,0,0,0,0,0,0,0,0
2,34.0,0.108993,0.486681,0.985791,-0.821243,-0.655336,1.486994,1.382454,2.443594,-1.525087,...,0,0,0,0,0,0,0,1,0,0
3,20.0,0.544966,0.407262,0.325088,0.118492,0.465238,1.486994,1.619138,0.284678,2.288477,...,0,0,0,0,0,0,0,0,0,0
4,26.0,0.544966,1.400007,1.335575,0.055843,-0.655336,0.436910,0.236406,0.457925,-0.324130,...,0,0,0,0,0,0,1,0,0,0


In [4]:
super_model_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 224 entries, 0 to 223
Data columns (total 58 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   TmScore                       224 non-null    float64
 1   O_1stD                        224 non-null    float64
 2   O_Tot_yd                      224 non-null    float64
 3   O_P_Yd                        224 non-null    float64
 4   O_R_Yd                        224 non-null    float64
 5   O_TO                          224 non-null    float64
 6   D_1stD                        224 non-null    float64
 7   D_Tot_Yd                      224 non-null    float64
 8   D_P_Yd                        224 non-null    float64
 9   D_R_Yd                        224 non-null    float64
 10  D_TO                          224 non-null    float64
 11  Home                          224 non-null    int64  
 12  Team_Arizona Cardinals        224 non-null    uint8  
 13  Team_

## MLP Regressor Model

In [6]:
# Data processing
X = super_model_data.drop('TmScore', axis=1).values
y = super_model_data['TmScore'].values.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.10)

MLP_model = MLPRegressor(hidden_layer_sizes=100, max_iter= 5000, random_state= 40, batch_size= 30, solver= 'lbfgs', 
                        activation= 'identity')

MLP_model.fit(X_train, y_train)

y_pred = MLP_model.predict(X_test)

print(mean_absolute_error(y_test, y_pred))


/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


5.804543996913073


## Gradient Boosting Regressor Model

In [7]:
GB_model = GradientBoostingRegressor(n_estimators= 2000, random_state= 20)

GB_model.fit(X_train, y_train)

y_pred = GB_model.predict(X_test)

print(mean_absolute_error(y_test, y_pred))

/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/utils/validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


4.748909903882049


## Random Forest Regressor Model

In [8]:
RF_model = RandomForestRegressor(n_estimators= 2000, random_state= 20)

RF_model.fit(X_train, y_train)

y_pred = RF_model.predict(X_test)

print(mean_absolute_error(y_test, y_pred))

/opt/anaconda3/envs/dev/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


4.942108695652173


In [ ]:
pd.options.display.max_rows = None
"""
importance = MLP_model.feature_importances_

feature_names = super_model_data.drop('TmScore', axis=1).columns

d = {'Var_Name': feature_names, 'Imp': importance}
df = pd.DataFrame(data=d)
df = df.sort_values(by= ['Imp'], ascending = False).reset_index(drop=True)
df.head(15)
"""

In [ ]:
# function that return the score of two teams playing

pd.options.display.max_columns = None

# Create playoff test dataset from season averages

def Score_Predictor(home_team, away_team, super_model):
    team1 = home_team
    team2 = away_team
    
    team1_data = super_model_data[com_data['Team'] == team1].drop('TmScore', axis=1).reset_index(drop=True)
    team2_data = super_model_data[com_data['Team'] == team2].drop('TmScore', axis=1).reset_index(drop=True)
    
    week_slice = slice(9,16)
    
    #1 Remove if no team names
    team1_test = pd.DataFrame(team1_data[week_slice].mean(axis=0)).T #select week to use as average
    team1_test
    opp_columns = team1_test.filter(like='Opp').columns
    
    team1_test[opp_columns] = 0
    team1_test['Opp_' + team2] = 1
    team1_test['Home'] = 1
    
    #2
    team2_test = pd.DataFrame(team2_data[week_slice].mean(axis=0)).T #select week to use as average
    opp_columns = team2_test.filter(like='Opp').columns
    
    team2_test[opp_columns] = 0
    team2_test['Opp_' + team1] = 1
    team2_test['Home'] = 0 #remove home field advantage
    
    # head to head matchup
    team1_test[['D_1stD','D_Tot_Yd','D_P_Yd','D_R_Yd','D_TO']] = team2_test[['O_1stD','O_Tot_yd','O_P_Yd','O_R_Yd','O_TO']]
    team2_test[['D_1stD','D_Tot_Yd','D_P_Yd','D_R_Yd','D_TO']] = team1_test[['O_1stD','O_Tot_yd','O_P_Yd','O_R_Yd','O_TO']]
    
    X_Playoff_test = pd.concat([team1_test, team2_test])
    X_Playoff_test.fillna(0, inplace = True) 
    
    scores = super_model.predict(X_Playoff_test)
    print(team1, "will score", round(scores[0], 1))
    print(team2, "will score", round(scores[1], 1))
    
    if scores[0] > scores[1]:
        winner = team1
    else:
        winner = team2
        
    print(winner, "are the WINNERS!!!")
    
    return scores, winner

### Road to the Superbowl
### Wild Card Round Matchups

In [12]:
playoff_teams = ['Buffalo Bills'{'Seed' : 3}, 'Pittsburgh Steelers' : {'Seed' : 7}]

SyntaxError: invalid syntax (<ipython-input-12-772b9727de60>, line 1)

In [10]:
[len(i) for i in playoff_teams]

NameError: name 'playoff_teams' is not defined

In [ ]:
, 
              'Kansas City Chiefs' : {'Abbrev':'KAN', 'Logo' : 'Logos/Chiefs.png', 'Seed' : 2}, 'Las Vegas Raiders' : {'Abbrev':'RAI', 'Logo' : 'Logos/Raiders.png', 'Seed' : 5},
              'Tennessee Titans' : {'Abbrev':'OTI', 'Logo' : 'Logos/Titans.png', 'Seed' : 1}, 'Los Angeles Rams' : {'Abbrev':'RAM', 'Logo' : 'Logos/Rams.png', 'Seed' : 4},
              'New England Patriots' : {'Abbrev':'NWE', 'Logo' : 'Logos/Patriots.png', 'Seed' : 6}, 'Tampa Bay Buccaneers' : {'Abbrev':'TAM', 'Logo' : 'Logos/Buccaneers.png', 'Seed' : 2}, 
              'San Francisco 49ers' : {'Abbrev':'SFO', 'Logo' : 'Logos/49ers.png', 'Seed' : 6}, 'Cincinnati Bengals' : {'Abbrev':'CIN', 'Logo' : 'Logos/Bengals.png', 'Seed' : 4}, 
              'Dallas Cowboys' : {'Abbrev':'DAL', 'Logo' : 'Logos/Cowboys.png', 'Seed' : 3}, 'Philadelphia Eagles' : {'Abbrev':'PHI', 'Logo' : 'Logos/Eagles.png', 'Seed' : 7}, 
              'Green Bay Packers' : {'Abbrev':'GNB', 'Logo' : 'Logos/Packers.png', 'Seed' : 1}, 'Arizona Cardinals' : {'Abbrev':'CRD', 'Logo' : 'Logos/Cardinals.png', 'Seed' : 5}}

In [ ]:
#https://www.vhv.rs/dpng/d/409-4095070_download-new-england-patriots-png-hd-pat-the.png
def Score_Predictor(home_team, away_team):
    team1 = home_team
    team2 = away_team
    
    
    scores1, afc_winner1 = Score_Predictor('Kansas City Chiefs', 'Pittsburgh Steelers')
    scores2, afc_winner2 = Score_Predictor('Buffalo Bills', 'New England Patriots')
    scores3, afc_winner3 = Score_Predictor('Cincinnati Bengals', 'Las Vegas Raiders')
    scores4, nfc_winner1 = Score_Predictor('Tampa Bay Buccaneers', 'Philadelphia Eagles')
    scores5, nfc_winner2 = Score_Predictor('Dallas Cowboys', 'San Francisco 49ers')
    scores6, nfc_winner3 = Score_Predictor('Los Angeles Rams', 'Arizona Cardinals')
    
    col2.subheader("Predictions")
    col2.write("AFC Game 1 Winner:")
    #col2.subheader(f"this is the winner of game 1:{scores1}")
    col2.image(teams_dict[afc_winner1]['Logo'])

    col2.write("AFC Game 2 Winner:")
    col2.image(teams_dict[afc_winner2]['Logo'])
    #col2.subheader(f"this is the winner of game 2:{winner2}")

    col2.write("AFC Game 3 Winner:")
    col2.image(teams_dict[afc_winner3]['Logo'])
    #col2.subheader(f"this is the winner of game 3:{winner3}")

    col2.write("NFC Game 1 Winner:")
    col2.image(teams_dict[nfc_winner1]['Logo'])
    #col2.subheader(f"this is the winner of game 1:{winner4}")

    col2.write("NFC Game 2 Winner:")
    col2.image(teams_dict[nfc_winner2]['Logo'])
    #col2.subheader(f"this is the winner of game 2:{winner5}")

    col2.write("NFC Game 3 Winner:")
    col2.image(teams_dict[nfc_winner3]['Logo'])
    #col2.subheader(f"this is the winner of game 3:{winner6}")

    col2.write("AFC Bye team: ")
    col2.image(teams_dict['Tennessee Titans']['Logo'])

    col2.write("NFC Bye team: ")
    col2.image(teams_dict['Green Bay Packers']['Logo'])
    
    # Original Code for round 2 added to the first prediction button
    
    afc_lowest = ''
    afc_team1 = ''
    afc_team2 = ''

    if (teams_dict[afc_winner1]['Seed'] > teams_dict[afc_winner2]['Seed']) and (teams_dict[afc_winner1]['Seed'] > teams_dict[afc_winner3]['Seed']):
        afc_lowest = afc_winner1
        afc_team1 = afc_winner2
        afc_team2 = afc_winner3
    elif (teams_dict[afc_winner2]['Seed'] > teams_dict[afc_winner1]['Seed']) and (teams_dict[afc_winner2]['Seed'] > teams_dict[afc_winner3]['Seed']):
        afc_lowest = afc_winner2
        afc_team1 = afc_winner1
        afc_team2 = afc_winner3
    elif (teams_dict[afc_winner3]['Seed'] > teams_dict[afc_winner1]['Seed']) and (teams_dict[afc_winner3]['Seed'] > teams_dict[afc_winner2]['Seed']):
        afc_lowest = afc_winner3
        afc_team1 = afc_winner1
        afc_team2 = afc_winner2
    
    
    nfc_lowest = ''
    nfc_team1 = ''
    nfc_team2 = ''

    if (teams_dict[nfc_winner1]['Seed'] > teams_dict[nfc_winner2]['Seed']) and (teams_dict[nfc_winner1]['Seed'] > teams_dict[nfc_winner3]['Seed']):
        nfc_lowest = nfc_winner1
        nfc_team1 = nfc_winner2
        nfc_team2 = nfc_winner3
    elif (teams_dict[nfc_winner2]['Seed'] > teams_dict[nfc_winner1]['Seed']) and (teams_dict[nfc_winner2]['Seed'] > teams_dict[nfc_winner3]['Seed']):
        nfc_lowest = nfc_winner2
        nfc_team1 = nfc_winner1
        nfc_team2 = nfc_winner3
    elif (teams_dict[nfc_winner3]['Seed'] > teams_dict[nfc_winner1]['Seed']) and (teams_dict[nfc_winner3]['Seed'] > teams_dict[nfc_winner2]['Seed']):
        nfc_lowest = nfc_winner3
        nfc_team1 = nfc_winner1
        nfc_team2 = nfc_winner2

    
    st.header("Divisional round")
    col1, col2 = st.columns(2)
    col1.subheader("Teams")
    col1.markdown("AFC Game 1")
    col1.image(teams_dict['Tennessee Titans']['Logo'], width = 200)
    col1.image(teams_dict[afc_lowest]['Logo'], width = 200)

    col1.markdown("AFC Game 2")
    col1.image(teams_dict[afc_team1]['Logo'], width = 200)
    col1.image(teams_dict[afc_team2]['Logo'], width = 200)

    col1.markdown("NFC Game 1")
    col1.image(teams_dict['Green Bay Packers']['Logo'], width = 200)
    col1.image(teams_dict[nfc_lowest]['Logo'], width = 200)

    col1.markdown("NFC Game 2")
    col1.image(teams_dict[nfc_team1]['Logo'], width = 200)
    col1.image(teams_dict[nfc_team2]['Logo'], width = 200)

    #button2 = st.button("Run Prediction 2")
    #if button2:
    st.write("prediction coming up...")
    scores7, winner7 = Score_Predictor('Tennessee Titans', afc_lowest)
    scores8, winner8 = Score_Predictor(afc_team1, afc_team2)
    scores9, winner9 = Score_Predictor('Green Bay Packers', nfc_lowest)
    scores10, winner10 = Score_Predictor(nfc_team1, nfc_team2)
   

    col2.subheader("Predictions")
    col2.write("AFC Game 1 Winner:")
    col2.image(teams_dict[winner7]['Logo'])

    col2.write("AFC Game 2 Winner:")
    col2.image(teams_dict[winner8]['Logo'])

    col2.write("NFC Game 1 Winner:")
    col2.image(teams_dict[winner9]['Logo'])

    col2.write("NFC Game 2 Winner:")
    col2.image(teams_dict[winner10]['Logo'])
        

    st.header("Conference championships")
    col1, col2 = st.columns(2)
    col1.subheader("teams")
    col1.markdown("AFC Final Game")
    col1.image(teams_dict[winner7]['Logo'], width = 200)
    col1.image(teams_dict[winner8]['Logo'], width = 200)

    col1.markdown("NFC Final Game")
    col1.image(teams_dict[winner9]['Logo'], width = 200)
    col1.image(teams_dict[winner10]['Logo'], width = 200)

        #button3 = st.button("Run Prediction 3")
        #if button3:
    st.write("prediction are being calculated...")
    scores11, winner11 = Score_Predictor(winner7, winner8)
    scores12, winner12 = Score_Predictor(winner9, winner10)

    col2.subheader("Predictions")
    col2.write("AFC Final Winner:")
    col2.image(teams_dict[winner11]['Logo'])
            
    col2.write("NFC Final Winner:")
    col2.image(teams_dict[winner12]['Logo'])
            
    

    st.header("Super Bowl")
    col1, col2 = st.columns(2)
    col1.image(teams_dict[winner11]['Logo'], width = 200)
    col1.image(teams_dict[winner12]['Logo'], width = 200)

          #  button4 = st.button("Run Prediction 4")
            #if button4:
    st.write("prediction are being calculated...")
    col2.write("Super Bowl Winner:")
    scores11, winner13 = Score_Predictor(winner11, winner12)
                
                
    col2.image(teams_dict[winner13]['Logo'])

In [ ]:
print(X_train.shape)

In [ ]:
wild_card_game = Score_Predictor('Philadelphia Eagles', 'Tampa Bay Buccaneers', MLP_model)

### NFC Playoff Wildcard Round 

In [ ]:
wc1 = Score_Predictor('Tampa Bay Buccaneers','Philadelphia Eagles', MLP_model)

In [ ]:
wc2 = Score_Predictor('Dallas Cowboys', 'San Francisco 49ers', MLP_model)

In [ ]:
scores, winner = Score_Predictor('Los Angeles Rams', 'Arizona Cardinals')

### AFC Playoff Wildcard Round 

In [ ]:
scores, winner = Score_Predictor('Kansas City Chiefs', 'Pittsburgh Steelers')

In [ ]:
scores, winner = Score_Predictor('Buffalo Bills', 'New England Patriots')

In [ ]:
scores, winner = Score_Predictor('Cincinnati Bengals', 'Las Vegas Raiders')

### NFC Playoff Divisional Round

In [ ]:
scores, winner = Score_Predictor('Green Bay Packers', 'Arizona Cardinals')

In [ ]:
scores, winner = Score_Predictor('Tampa Bay Buccaneers', 'Dallas Cowboys')

### AFC Playoff Divisional Round

In [ ]:
scores, winner = Score_Predictor('Tennessee Titans', 'New England Patriots')

In [ ]:
scores, winner = Score_Predictor('Kansas City Chiefs', 'Las Vegas Raiders')

### NFC Conference Championship

In [ ]:
scores, winner = Score_Predictor('Tampa Bay Buccaneers', 'Arizona Cardinals')

### AFC Conference Championship

In [ ]:
scores, winner = Score_Predictor('Tennessee Titans', 'Kansas City Chiefs')

### Suber Bowl

In [ ]:
scores, winner = Score_Predictor('Kansas City Chiefs', 'Arizona Cardinals')

### NFC Playoff Picture

1. Green Bay Packers (13-4), champions, NFC North. The Packers clinched the No. 1 seed, the lone bye and the Lambeau home-field advantage in the NFC playoffs with their win over the Vikings in Week 17, matching their win total of the previous two seasons with Aaron Rodgers under Matt LaFleur with one more game left in the new schedule. They lost in Week 18 while resting Rodgers and other regulars in the second half against the Lions.

2. Tampa Bay Buccaneers (13-4), champions, NFC South. The Buccaneers beat the Panthers and moved up to No. 2 because the Rams, to whom they lost in Week 3, lost to the 49ers. They finished behind the Packers because of a lesser confference record and ahead of the Cowboys, up a full game and a head-to-head tiebreaker from Week 1.

3. Dallas Cowboys (12-5), champions, NFC East. The Cowboys rebounded to rout the resting Eagles in Week 18 after they lost to the Cardinals in a failed comeback in Week 17. They were able to jump the losing Rams because of winning the conference-record tiebreaker over them.

4. Los Angeles Rams (12-5), champions, NFC West. The Rams failed to beat the 49ers in Week 18 but still took back the division crown with the Cardinals losing another West matchup to the Seahawks at home. They cost themselves a No. 2 seed and now need to play a third game against the Cardinals.

5. Arizona Cardinals (11-6), second place, NFC West. The Cardinals could have won the division with the Rams losing but they also lost to the Seahawks. They will settle for the top wild-card spot and a rematch with the Rams in the wild-card playoffs.

6. San Francisco 49ers (10-7), third place, NFC West. The 49ers locked down the second wild card by beating the Rams in overtime in Week 18. They held off the winning Saints and got up a full game on the losing Eagles, whom they also beat in Week 2.

7. Philadelphia Eagles (9-8), second place, NFC East. The Eagles rested players against the Cowboys in Week 18 after they had already clinched a wild-card spot in Week 17. They stayed ahead of the winning Saints because of beating that team in Week 11.

### AFC Playoff Picture

1. Tennessee Titans (12-5), champions, AFC South. The Titans held on against the Texans in Week 18 to stay ahead of the Chiefs and clinch the No. 1 seed. They have home-field advantage in the AFC playoffs and the lone bye. The conference road to Super Bowl 56 will go through Nashviille.

2. Kansas City Chiefs (12-5), champions, AFC West. The Chiefs will settle for the No. 2 seed after beating the Broncos in Week 18 because they lost head-to-head to the Titans in Week 7. They did win Super Bowl 54 coming from this position behind the Ravens.

3. Buffalo Bills (11-6), champions, AFC East. The Bills won the division and kept this seed with a home win over the Jets. They finish a game behind the co-AFC leaders and a game ahead of the Bengals after being the AFC's No. 2 in last year's playoffs.

4. Cincinnati Bengals (10-7), champions, AFC North. The Bengals rested key players because of injuries and other reasons and lost without Joe Burrow and Joe Mixon at the Browns in Week 18. They were set to stay here behind the East champions.

5. Las Vegas Raiders (10-7), second place, AFC West. The Raiders moved into playoff position with the Colts, whom they beat in Week 17, being upset by the Jaguars in Week 18. They earned a playoff berth with the wild overtime win over the Chargers on Sunday night.

6. New England Patriots (10-7), second place, AFC East. The Patriots lost to the Dolphins in Week 18, but they had already lost the East title when the Bills beat the Jets. They dropped to No. 6 with the Raiders beating the Chargers on Sunday night because of losing the tiebreaker.

7. Pittsburgh Steelers (9-7-1), second place, AFC North. The Steelers got into the playoffs to extend the career of Ben Roethlisberger by beating the Ravens in Week 18 while the Jaguars beat the Colts and the Raiders beat the Chargers.

